In [ ]:
# Reference:https://www.analyticsvidhya.com/blog/2019/08/comprehensive-guide-language-model-nlp-python-code/

In [64]:
from nltk.corpus import reuters
from nltk import bigrams, trigrams 
from nltk.lm.preprocessing import pad_both_ends
from nltk.lm.preprocessing import flatten


from nltk.tokenize import sent_tokenize, word_tokenize
from collections import defaultdict


In [ ]:
from nltk.corpus import reuters

In [7]:
with open('corpus') as f:
    corpus = f.read()

In [8]:
corpus_list = []
for line in corpus.split('\n'):
    corpus_list.append(line)

In [9]:
label_length = len('__label__2')
y_list = []
X_list = []
for document in corpus_list:
    y_list.append(document[:label_length])
    X_list.append(document[label_length+1:])

### Bigram

In [35]:
sentence = 'i ate an apple'
sentence_word = word_tokenize(sentence)
dic = {}
new_list = [(w1, w2) for w1, w2 in list(bigrams(sentence_word, pad_right = True, pad_left = True))]
collections.Counter(new_list)

Counter({(None, 'i'): 1,
         ('i', 'ate'): 1,
         ('ate', 'an'): 1,
         ('an', 'apple'): 1,
         ('apple', None): 1})

In [67]:
bigram_dict = defaultdict(lambda: defaultdict(int))


for paragraph in X_list:
    for sentence in sent_tokenize(paragraph):
        sentence_word = word_tokenize(sentence)
        for w1, w2 in bigrams(sentence_word, pad_right = True, pad_left = True):
            bigram_dict[w1][w2] += 1


In [73]:
for w1 in bigram_dict:
    total_w1_cnt = sum(bigram_dict[w1].values())
    for w2 in bigram_dict[w1]:
        bigram_dict[w1][w2] /= total_w1_cnt        

In [74]:
bigram_dict['even']['if']

0.0494728304947283

In [106]:
# a function to get the next word:
def get_next_word(starting_word):
    most_freq_word = sorted([(next_word, prob) for (next_word, prob) in bigram_dict[starting_word].items()], 
                            key = lambda x: -x[1])[0][0]
    return most_freq_word

In [107]:
get_next_word('even')

'though'

In [108]:
get_next_word('eat')

'it'

In [141]:
# a function to get the most likely sentence:
def get_most_likely_sentence(starting_word):
    n_iteration = 0
    sentence = [starting_word]
    while n_iteration < 10:
        
        next_word = get_next_word(starting_word)
        
        n_iteration += 1
        
        sentence.append(next_word)
        
        if next_word == None:
            return sentence
        
        starting_word = next_word
    
    final_sentence = ''.join([word for word in sentence if word])
  
    return final_sentence


In [142]:
get_most_likely_sentence('even')

['even', 'though', ',', 'and', 'the', 'book', '.', None]

In [143]:
get_most_likely_sentence('oddly')

['oddly', 'into', 'the', 'book', '.', None]

In [125]:
# a function to generate random sentence

import random

def get_sentence(starting_word):

    text = [starting_word]
    sentence_finished = False

    n_iteration = 0 

    while not sentence_finished:
        threshold = 0.005
        
        for word in bigram_dict[text[-1]].keys():
            n_iteration += 1
            
            if n_iteration > 30:
                break

            if bigram_dict[text[-1]][word] > threshold:
                text.append(word)
                break

            if text[-1] == None:
                sentence_finished = True
                break

        if len(text) > 10:
            sentence_finished = True
            break

#     return ' '.join([t for t in text if t])
    return text


In [126]:
get_sentence('even')

['even', 'for', 'the', 'best', 'music', '!', None]

### Trigram

In [146]:
sentence = 'i ate an apple'
sentence_word = word_tokenize(sentence)
sentence_word_trigram = list(trigrams(sentence_word, pad_left = True, pad_right = True))
print(sentence_word_trigram)

[(None, None, 'i'), (None, 'i', 'ate'), ('i', 'ate', 'an'), ('ate', 'an', 'apple'), ('an', 'apple', None), ('apple', None, None)]


In [147]:
trigram_dict = defaultdict(lambda: defaultdict(int))

for paragraph in X_list:
    for sentence in sent_tokenize(paragraph):
        sentence_word = word_tokenize(sentence)
        for w1, w2, w3 in trigrams(sentence_word, pad_right = True, pad_left = True):
            trigram_dict[(w1, w2)][w3] += 1


In [151]:
# calculate probability

for w1, w2 in trigram_dict:
    total_cnt = sum(trigram_dict[(w1, w2)].values())
    for w3 in trigram_dict[(w1,w2)]:
        trigram_dict[(w1, w2)][w3] /= total_cnt
    
# trigram_dict[('I', 'would')]

In [152]:
def get_next_word_trigram(w1, w2):
    next_word = sorted([(w3, prob) for (w3, prob) in trigram_dict[(w1,w2)].items()], key = lambda x: -x[1])[0][0]
    return next_word
    

In [153]:
get_next_word_trigram('I', 'would')

'recommend'

In [154]:
import numpy as np
import pandas as pd
from keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import LSTM, Dense, GRU, Embedding
from keras.callbacks import EarlyStopping, ModelCheckpoint

Using TensorFlow backend.


In [156]:
def create_seq(text):
    length = 30
    sequences = list()
    for i in range(length, len(text)):
        # select sequence of tokens
        seq = text[i-length:i+1]
        # store
        sequences.append(seq)
    print('Total Sequences: %d' % len(sequences))
    return sequences



In [162]:
X_list[0]

'Stuning even for the non-gamer: This sound track was beautiful! It paints the senery in your mind so well I would recomend it even to people who hate vid. game music! I have played the game Chrono Cross but out of all of the games I have ever played it has the best music! It backs away from crude keyboarding and takes a fresher step with grate guitars and soulful orchestras. It would impress anyone who cares to listen! ^_^'

In [170]:
# create sequences   
sequences = create_seq(X_list[0])

Total Sequences: 396


In [171]:
sequences[:10]

['Stuning even for the non-gamer:',
 'tuning even for the non-gamer: ',
 'uning even for the non-gamer: T',
 'ning even for the non-gamer: Th',
 'ing even for the non-gamer: Thi',
 'ng even for the non-gamer: This',
 'g even for the non-gamer: This ',
 ' even for the non-gamer: This s',
 'even for the non-gamer: This so',
 'ven for the non-gamer: This sou']

In [163]:
# create a character mapping index
chars = sorted(list(set(X_list[0])))
mapping = dict((c, i) for i, c in enumerate(chars))

def encode_seq(seq):
    sequences = list()
    for line in seq:
        # integer encode line
        encoded_seq = [mapping[char] for char in line]
        # store
        sequences.append(encoded_seq)
    return sequences

# encode the sequences
sequences = encode_seq(sequences)

In [165]:
from sklearn.model_selection import train_test_split

# vocabulary size
vocab = len(mapping)
sequences = np.array(sequences)
# create X and y
X, y = sequences[:,:-1], sequences[:,-1]
# one hot encode y
y = to_categorical(y, num_classes=vocab)
# create train and validation sets
X_tr, X_val, y_tr, y_val = train_test_split(X, y, test_size=0.1, random_state=42)

print('Train shape:', X_tr.shape, 'Val shape:', X_val.shape)

Train shape: (356, 30) Val shape: (40, 30)


In [166]:
# define model
model = Sequential()
model.add(Embedding(vocab, 50, input_length=30, trainable=True))
model.add(GRU(150, recurrent_dropout=0.1, dropout=0.1))
model.add(Dense(vocab, activation='softmax'))
print(model.summary())

# compile the model
model.compile(loss='categorical_crossentropy', metrics=['acc'], optimizer='adam')
# fit the model
model.fit(X_tr, y_tr, epochs=100, verbose=2, validation_data=(X_val, y_val))

Instructions for updating:
Colocations handled automatically by placer.
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 30, 50)            1650      
_________________________________________________________________
gru_1 (GRU)                  (None, 150)               90450     
_________________________________________________________________
dense_1 (Dense)              (None, 33)                4983      
Total params: 97,083
Trainable params: 97,083
Non-trainable params: 0
_________________________________________________________________
None
Instructions for updating:
Use tf.cast instead.
Train on 356 samples, validate on 40 samples
Epoch 1/100
 - 1s - loss: 3.4658 - acc: 0.1601 - val_loss: 3.4287 - val_acc: 0.1750
Epoch 2/100
 - 0s - loss: 3.1637 - acc: 0.1910 - val_loss: 3.3001 - val_acc: 0.1750
Epoch 3/100
 - 0s - loss: 3.0493 - acc: 0.

Epoch 82/100
 - 0s - loss: 0.1567 - acc: 0.9803 - val_loss: 4.5709 - val_acc: 0.2750
Epoch 83/100
 - 0s - loss: 0.1481 - acc: 0.9888 - val_loss: 4.6624 - val_acc: 0.2250
Epoch 84/100
 - 0s - loss: 0.1420 - acc: 0.9860 - val_loss: 4.7283 - val_acc: 0.2750
Epoch 85/100
 - 0s - loss: 0.1302 - acc: 0.9860 - val_loss: 4.7703 - val_acc: 0.2500
Epoch 86/100
 - 0s - loss: 0.1240 - acc: 0.9888 - val_loss: 4.7983 - val_acc: 0.2250
Epoch 87/100
 - 0s - loss: 0.1190 - acc: 0.9944 - val_loss: 4.8010 - val_acc: 0.2250
Epoch 88/100
 - 0s - loss: 0.1117 - acc: 0.9916 - val_loss: 4.8301 - val_acc: 0.2500
Epoch 89/100
 - 0s - loss: 0.1006 - acc: 0.9972 - val_loss: 4.8344 - val_acc: 0.2500
Epoch 90/100
 - 0s - loss: 0.0982 - acc: 0.9888 - val_loss: 4.8904 - val_acc: 0.2500
Epoch 91/100
 - 0s - loss: 0.0886 - acc: 0.9944 - val_loss: 4.9192 - val_acc: 0.2250
Epoch 92/100
 - 0s - loss: 0.0881 - acc: 1.0000 - val_loss: 4.9270 - val_acc: 0.2250
Epoch 93/100
 - 0s - loss: 0.0932 - acc: 0.9944 - val_loss: 4.907

In [168]:
# generate a sequence of characters with a language model
def generate_seq(model, mapping, seq_length, seed_text, n_chars):
    in_text = seed_text
    # generate a fixed number of characters
    for _ in range(n_chars):
        # encode the characters as integers
        encoded = [mapping[char] for char in in_text]
        # truncate sequences to a fixed length
        encoded = pad_sequences([encoded], maxlen=seq_length, truncating='pre')
        # predict character
        yhat = model.predict_classes(encoded, verbose=0)
        # reverse map integer to character
        out_char = ''
        for char, index in mapping.items():
            if index == yhat:
                out_char = char
                break
        # append to input
        in_text += char
    return in_text
